(sec_tskit_no_mutations)=

# Do you really need mutations?

Maybe not! This tutorial takes you through a number of common tasks 
and shows how to do things by working directly with the underlying 
topology.

## Background

Why is it better to avoid using mutations if we don't have to?

## Computing statistics